### Categoricals
- almost all machine learning algorithms use numeric data for data analysis. Therefore, we need to convert the categorical data into numeric form
- there are different functions available in different packages. Here I am going to try some

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Unlike R, python doesn't have a factor function to convert categorical data to numeric data for machine learning.
So, we have to do it using different approach!
Here is our dataset

In [4]:
df = pd.DataFrame({'factor1':['a','a','a','b','c','c','c'], 'factor2':['d','a','d','b','c','d','c'], 'num1':range(1,8)})
print(df)
print(df.info())

  factor1 factor2  num1
0       a       d     1
1       a       a     2
2       a       d     3
3       b       b     4
4       c       c     5
5       c       d     6
6       c       c     7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
factor1    7 non-null object
factor2    7 non-null object
num1       7 non-null int64
dtypes: int64(1), object(2)
memory usage: 248.0+ bytes
None


#### First approach: using LabelEncoder() from sklearn.preprocessing! 
- The coding is in integer! However, such integer representation can not be used directly with scikit-learn estimators, as these expect continuous input, and would interpret the categories as being ordered, which is often not desired (i.e. the set of browsers was ordered arbitrarily)

In [5]:
le = preprocessing.LabelEncoder()
df1 = df.copy()

cat_columns = df1.select_dtypes(['object']).columns

# convert categorical data to numeric
for cat in df1.columns.values:
    if df1[cat].dtype == 'object':
        df1[cat] = le.fit_transform(df1[cat])
        
#df1.factor2 = le.fit_transform(df1.factor2)
print(df1)

# convert it from int to float. However, once it is converted to float, you won't be able to convert it back to categorical!
df1_copy = df1.copy()

for idx in range(len(cat_columns)):
    df1_copy[cat_columns[idx]] = df1_copy[cat_columns[idx]].astype(np.float32)
    
#df1_copy.factor2 = df1_copy.factor2.astype(np.float32)
print(df1_copy)

# convert it back to the categorical. ==> to run this, you need to reload the df from the previous cell
df1.factor2 = le.inverse_transform(df1.factor2)
print(df1)

   factor1  factor2  num1
0        0        3     1
1        0        0     2
2        0        3     3
3        1        1     4
4        2        2     5
5        2        3     6
6        2        2     7
   factor1  factor2  num1
0      0.0      3.0     1
1      0.0      0.0     2
2      0.0      3.0     3
3      1.0      1.0     4
4      2.0      2.0     5
5      2.0      3.0     6
6      2.0      2.0     7
   factor1 factor2  num1
0        0       d     1
1        0       a     2
2        0       d     3
3        1       b     4
4        2       c     5
5        2       d     6
6        2       c     7


#### Second: is using select_dtypes and cat.codes() function in pandas

In [6]:
df4 = df.copy()

# need to convert object type to category type
for cat in df4.columns.values:
    #print(df4[cat].dtype)
    if df4[cat].dtype == 'object':
        df4[cat] = df4[cat].astype('category')
    
cat_columns = df4.select_dtypes(['category']).columns
print(cat_columns)

df5 = df4.copy()
# the following will convert categorical column to int only
df4[cat_columns] = df4[cat_columns].apply(lambda x: x.cat.codes)
print(df4)

# however, to convert to float, you have to do the following:
df5[cat_columns] = df5[cat_columns].apply(lambda x: x.cat.codes.astype(np.float32))
print(df5)



Index(['factor1', 'factor2'], dtype='object')
   factor1  factor2  num1
0        0        3     1
1        0        0     2
2        0        3     3
3        1        1     4
4        2        2     5
5        2        3     6
6        2        2     7
   factor1  factor2  num1
0      0.0      3.0     1
1      0.0      0.0     2
2      0.0      3.0     3
3      1.0      1.0     4
4      2.0      2.0     5
5      2.0      3.0     6
6      2.0      2.0     7


#### Third, use OneHotEncoder() from sklearn.preprocessing. 
- Need to use DictVectorizer from sklearn.feature_extraction. The following article
http://stackoverflow.com/questions/17469835/one-hot-encoding-for-machine-learning gives a very good explaination regarding 'one hot encoding'. 
https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science

In [7]:
le  = LabelEncoder()

df6 = df.copy()
print(df6)

# list of cat columns
cat_cols = []
for col in df6.columns.values:
    if df6[col].dtype == 'object':
        df6[col] = le.fit_transform(df6[col])
        cat_cols.append(df6.columns.get_loc(col))   # get cat column index
        
print(df6)
print(cat_cols)

# create OneHotEncoder instance with categorical feature indices included
ohe = OneHotEncoder(categorical_features=np.array(cat_cols))

# now convert the df6 using  one hot encoder as it only takes matrix of int as input
df6 = ohe.fit_transform(df6).toarray()
print(ohe.n_values_)
print(ohe.feature_indices_)
print(ohe.active_features_)
print(df6)

  factor1 factor2  num1
0       a       d     1
1       a       a     2
2       a       d     3
3       b       b     4
4       c       c     5
5       c       d     6
6       c       c     7
   factor1  factor2  num1
0        0        3     1
1        0        0     2
2        0        3     3
3        1        1     4
4        2        2     5
5        2        3     6
6        2        2     7
[0, 1]
[3 4]
[0 3 7]
[0 1 2 3 4 5 6]
[[ 1.  0.  0.  0.  0.  0.  1.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  2.]
 [ 1.  0.  0.  0.  0.  0.  1.  3.]
 [ 0.  1.  0.  0.  1.  0.  0.  4.]
 [ 0.  0.  1.  0.  0.  1.  0.  5.]
 [ 0.  0.  1.  0.  0.  0.  1.  6.]
 [ 0.  0.  1.  0.  0.  1.  0.  7.]]


#### Fourth: using sklearn.feature_extraction's DictVectorizer

In [11]:
from sklearn.feature_extraction import DictVectorizer

df2 = df.copy()    # deep copy!
print(df2)
print("\nDisplay as dictionary\n", df2.to_dict('records'))

dVec = DictVectorizer()
df2 = dVec.fit_transform(df2.to_dict('records')).toarray()
print("\nThe converted ones\n", df2)
print(dVec.get_feature_names())

  factor1 factor2  num1
0       a       d     1
1       a       a     2
2       a       d     3
3       b       b     4
4       c       c     5
5       c       d     6
6       c       c     7

Display as dictionary
 [{'factor1': 'a', 'num1': 1, 'factor2': 'd'}, {'factor1': 'a', 'num1': 2, 'factor2': 'a'}, {'factor1': 'a', 'num1': 3, 'factor2': 'd'}, {'factor1': 'b', 'num1': 4, 'factor2': 'b'}, {'factor1': 'c', 'num1': 5, 'factor2': 'c'}, {'factor1': 'c', 'num1': 6, 'factor2': 'd'}, {'factor1': 'c', 'num1': 7, 'factor2': 'c'}]

The converted ones
 [[ 1.  0.  0.  0.  0.  0.  1.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  2.]
 [ 1.  0.  0.  0.  0.  0.  1.  3.]
 [ 0.  1.  0.  0.  1.  0.  0.  4.]
 [ 0.  0.  1.  0.  0.  1.  0.  5.]
 [ 0.  0.  1.  0.  0.  0.  1.  6.]
 [ 0.  0.  1.  0.  0.  1.  0.  7.]]
['factor1=a', 'factor1=b', 'factor1=c', 'factor2=a', 'factor2=b', 'factor2=c', 'factor2=d', 'num1']


#### Fifth: use get_dummies() from Pandas. 
However, It converts a column of categorical variables (0,1,...k) to dummy k columns of (0, 1), that is binary (in float). Therefore, it is also called one hot encoding!

In [12]:
df3 = df.copy()
print(df3)

#df3.factor2 = pd.get_dummies(df3.factor2)
df3 = pd.get_dummies(df3)
print(df3)


  factor1 factor2  num1
0       a       d     1
1       a       a     2
2       a       d     3
3       b       b     4
4       c       c     5
5       c       d     6
6       c       c     7
   num1  factor1_a  factor1_b  factor1_c  factor2_a  factor2_b  factor2_c  \
0     1        1.0        0.0        0.0        0.0        0.0        0.0   
1     2        1.0        0.0        0.0        1.0        0.0        0.0   
2     3        1.0        0.0        0.0        0.0        0.0        0.0   
3     4        0.0        1.0        0.0        0.0        1.0        0.0   
4     5        0.0        0.0        1.0        0.0        0.0        1.0   
5     6        0.0        0.0        1.0        0.0        0.0        0.0   
6     7        0.0        0.0        1.0        0.0        0.0        1.0   

   factor2_d  
0        1.0  
1        0.0  
2        1.0  
3        0.0  
4        0.0  
5        1.0  
6        0.0  


#### Sixth: using pandas.factorize

In [23]:
df6 = df.copy()
print("original data set\n", df6)

for col in df6.columns:
    if df6[col].dtype == 'object':
        df6[col], unique = pd.factorize(df6[col])
    
print(df6)

original data set
   factor1 factor2  num1
0       a       d     1
1       a       a     2
2       a       d     3
3       b       b     4
4       c       c     5
5       c       d     6
6       c       c     7
   factor1  factor2  num1
0        0        0     1
1        0        1     2
2        0        0     3
3        1        2     4
4        2        3     5
5        2        0     6
6        2        3     7


In [24]:
dff = pd.DataFrame([['rick','young'],['phil','old'],['john','teenager']],columns=['name','age-group'])
print(dff.describe())
print(dff.dtypes)

pd.get_dummies(dff)

        name age-group
count      3         3
unique     3         3
top     rick     young
freq       1         1
name         object
age-group    object
dtype: object


,name_john,name_phil,name_rick,age-group_old,age-group_teenager,age-group_young
0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0
